In [1]:
import json
import pandas as pd
import requests as rq
import os
import time
from bs4 import BeautifulSoup
import re
import numpy as np

###### 房源列表

In [2]:
out_path=r"D:\ctripdata"

In [3]:
def out_put(href,title,corpus):
    f=open(out_path+'\\'+title+r'.txt','w',encoding='utf-8')
    f.write(href+'\n')
    f.write(corpus)
    f.close()
#     print('success!')

In [5]:
# 民宿所在城市
# city=r'chengdu10'
city=r'nanjing9'
# city=r'shanghai23'
# citylist=['beijing48','shanghai23','chengdu10','guangzhou45','hangzhou7','nanjing9','qingdao8','chongqing22','xian17','suzhou11','wuhan55','xiamen33','taibei4005']
citylist=['lijiang12','dali36','kunming31','zhengzhou167','haerbin64','jinan19','nanchang346','zhangjiajie342','huangshan6','sanya1','jiaxing110','qinhuangdao65','zhoushan20','shenzhen49','guilin32','luoyang168']
prices=["0-200","200-250","250-300","300-350","350-400","500-10000"] # 200--250 修改

In [7]:
urllist=[]
# 构造爬取的urllist，网页限制最多爬60页
for city in citylist:
    for price in prices:
        for p in range(60):
            url="https://inn.ctrip.com/onlineinn/newlist/"+city+r"/?"+"m="+price+"&d1=2020-09-24&d2=2020-09-25"+"&s=1&p="+str(p)
            urllist.append(url)

In [8]:
out_put('urllist','urllist2','\n'.join(urllist))

In [24]:
urllist=[]
with open(out_path+'\\'+'urllist'+r'.txt','r',encoding='utf-8') as f:
    for l in f.readlines():
        urllist.append(l.strip())

In [29]:
response=rq.get(url)
texts=response.text
soup=BeautifulSoup(texts,'lxml')

# 找到带有unitid的所有房源信息
# houses=soup.find_all('script')[9].text

In [70]:
# 构造爬取的urllist，修改200-250
urllist=[]
for city in citylist:
    price='200-250'
    for p in range(60):
        url="https://inn.ctrip.com/onlineinn/newlist/"+city+r"/?"+"m="+price+"&d1=2020-09-24&d2=2020-09-25"+"&s=1&p="+str(p)
        urllist.append(url)

In [18]:
# 所有的房源信息列表
# allitems=[]
stop=0
for i in range(stop,len(urllist)):
# for i in range(len(urllist)):
    url=urllist[i]
    response=rq.get(url)
    texts=response.text
    if response.status_code==200:
        out_put(url,str(url.split('/')[5])+'_'+str(url.split('=')[1].replace('&d1',''))+'_'+str(url.split('=')[-1]),texts)
    else:
        print("访问出错")
        break
    soup=BeautifulSoup(texts,'lxml')

    # 找到带有unitid的所有房源信息
    # houses=soup.find_all('script')[9].text
    if len(soup.find_all('script',text=re.compile("unitId"))) > 0:
        houses=soup.find_all('script',text=re.compile("unitId"))[0].text
#       用分号分隔：houses=re.sub('__NEXT_DATA__ = ','',houses.split(';')[2])
        houses=houses.split('"props":')[1].split(',"isSSR":')[0]  # 分隔两次 获取props后的字典内容
        hjson=json.loads(houses)

#         items=hjson['props']['pageProps']['initialState']['items']
        items=hjson['pageProps']['initialState']['items']
        allitems.extend(items)
    else:
        continue
    if i%500==0:
        time.sleep(60)

In [85]:
allitems[0].keys()

dict_keys(['unitId', 'unitName', 'houseTypeName', 'pictureList', 'defaultPicture', 'pictureCount', 'logoUrl', 'geoCoordSysType', 'longitude', 'latitude', 'address', 'allowBooking', 'unitInfor', 'inTns', 'active', 'activityInfo', 'activityId', 'failType', 'productPrice', 'finalPrice', 'addPeopleAmount', 'cleaningAmount', 'priceMissingText', 'cityId', 'cityName', 'districtName', 'chineseLandlord', 'quickBooking', 'promotionPicUrl', 'smallPictureUrl', 'unitSummeries', 'adverUnit', 'advertUnitType', 'great', 'expressBooking', 'priceTags', 'rba', 'qualityCertification', 'checkInDate', 'checkOutDate', 'unitTags', 'distanceTip', 'showHouseVideo', 'showHouseVR', 'topListTag', 'urgencyTip', 'houseTags', 'finalPriceDisplayType', 'priceTipBadge', 'displayOrigionPrice', 'houseListTagImage', 'productId', 'landlordLevelUrl', 'productPriceConstract', 'promoText', 'rankScore', 'rankScore2', 'distance', 'extends4Ctrip', 'hotleId', 'recommendedGuests', 'bedCount', 'housingType', 'theHouseType', 'comment

In [91]:
# 需要获取的房源的属性
ikeys=['unitId', 'unitName','houseTypeName','pictureCount','longitude', 'latitude','address','unitInfor', 'productPrice', 'finalPrice', 'cityId','cityName','districtName','productId','rankScore','rankScore2','extends4Ctrip','distance', 'hotleId', 'recommendedGuests', 'bedCount', 'housingType', 'theHouseType', 'commentScore','preloadDetail', 'distanceDesc', 'rname']

In [106]:
# 将数据写入dataframe
for item in allitems:
    for key in ikeys:
        if item[key]!=None:
            ctrips.at[i,key]=str(item[key])
        else:
            ctrips.at[i,key]='none'
#   房源的评分和评论数
    for dicts in item['unitSummeries']:
        summary="none"
        if dicts['textType']=='COMMENT':
            summary=dicts['text']
            break
    if summary == "none":
        ctrips.at[i,'score']='none'
        ctrips.at[i,'numcomments']='none'
    else:
        summary=summary.split('/')
        ctrips.at[i,'score']=summary[0].replace("分","")
        ctrips.at[i,'numcomments']=summary[1].replace("点评","")
    i+=1

In [108]:
crtips_clean=ctrips.drop_duplicates()

In [218]:
ctrips.to_csv(out_path+r'\ctrips_13_6'+r'.csv',index=None,encoding='utf_8_sig')

In [111]:
crtips_clean.to_csv(out_path+r'\ctrips_13_6'+r'.csv',index=None,encoding='utf_8_sig')